# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import time
from functools import reduce

!pip install html5lib -v

Using pip 22.2.2 from /home/fame/anaconda3/lib/python3.9/site-packages/pip (python 3.9)


# Data

## Define SeasonData Object

In [2]:
class SeasonsData:
    def __init__(self, years):
        self.years = years
        self.season_tables = {year: self.get_season_tables(year) for year in self.years}
        self.season_datasets = {year: self.get_season_datasets(self.season_tables[year]) for year in self.years}
        self.complete_dataset =  self.get_complete()
        self.shape = self.complete_dataset.shape
        
    def get_season_tables(self, year):
        """
        Get each table from hockey-reference
        season databases
        """

        # -- Define URLS --

        ref_url = 'https://www.hockey-reference.com'

        basic_ext = f'/leagues/NHL_{year}_skaters.html'
        adv_ext = f'/leagues/NHL_{year}_skaters-advanced.html'
        shift_ext = f'/leagues/NHL_{year}_skaters-time-on-ice.html'
        
        #### HAD TO REMOVE MISC BECAUSE IT WASNT TRACKED FOR MOST YEARS
        # misc_ext = f'/leagues/NHL_{year}_skaters-misc.html'

        url_names = ['basic', 'adv', 'shift',]
        url_ext = [basic_ext, adv_ext, shift_ext]

        # --- Get Tables ---

        raw_tables = {}
        print(f'Grabbing {year} season data')
        for name, ext in zip(url_names, url_ext):
            req = False
            fails = 0
            while req == False:
                sleep_time = 2**fails
                try:
                    df = pd.read_html(ref_url+ext)[0].apply(lambda x: x.astype(str).str.upper())
                    req = True
                except:
                    fails += 1
                    print(fails, 'fails')
                    time.sleep(sleep_time)
                    pass
            li = list(df.columns)
            col = [ x[1] for x in li ]
            df.columns = col
            df = df[df['Rk'] != 'RK'].reset_index(drop=True).sort_values(by='Player')
            tot_tm = df[df['Tm'] == 'TOT']
            no_dupes = df.drop_duplicates(subset='Player', keep=False)
            df = pd.concat([no_dupes, tot_tm])
            df_dict = {name: df}
            raw_tables.update(df_dict)
            print(f'- Grabbed {name}')
        
        return raw_tables

    def get_season_datasets(self, raw_tables):
        """
        Cleans columns of each dataframe and 
        merges all dataframes into one complete 
        dataframe
        """

        # Basic Dataframe
        basic_df = raw_tables['basic'].drop(['FO%'], axis=1)
        basic_df.columns =    [
                                'Rk', 'Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A', 'PTS', '+/-', 'PIM',
                                'PS', 'EV_G', 'PP_G', 'SH_G', 'GW_G', 'EV_A', 'PP_A', 'SH_A', 'S', 'S%', 'TOI',
                                'ATOI', 'BLK', 'HIT', 'FOW', 'FOL'
                            ]
        raw_tables['basic'] = basic_df

        # Adv Dataframe
        adv_df = raw_tables['adv'].drop(['GP', 'Age', 'Rk', 'Pos'], axis=1)
        try:
            adv_df = adv_df.drop(['E+/-'], axis=1)
        except:
            pass
        adv_df.columns =    [
                                'Player', 'Tm', 'CF', 'CA', 'CF%', 'CF% rel',
                                'FF', 'FA', 'FF%', 'FF% rel', 'oiSH%', 'oiSV%', 'PDO', 'oZS%', 'dZS%',
                                'TOI/60', 'TOI(EV)', 'TK', 'GV', 'SAtt.', 'Thru%'
                            ]
        raw_tables['adv'] = adv_df

        # Shift Dataframe
        shift_df = raw_tables['shift'].drop([
            'GP', 'Unnamed: 6_level_1', 'Unnamed: 11_level_1', 'Unnamed: 16_level_1', 'Rk', 'Pos'
        ], axis=1)
        shift_df.columns =  [
                                'Player', 'Tm', 'Shift', 'TOI_EVEN',
                                'CF% Rel_EVEN', 'GF/60_EVEN', 'GA/60_EVEN', 'TOI_PP', 'CF% Rel_PP',
                                'GF/60_PP', 'GA/60_PP', 'TOI_SH', 'CF% Rel_SH', 'GF/60_SH',
                                'GA/60_SH'
                            ]
        raw_tables['shift'] = shift_df
        
        #### HAD TO REMOVE MISC BECAUSE IT WASNT TRACKED FOR MOST YEARS
        # Misc Dataframe
        # misc_df = raw_tables['misc'].drop([
        #     'GP', 'Age', 'Made', 'Miss', 'Pct.', 'Rk', 'Pos', '+/-', 'PS'
        # ], axis=1)
        # try:
        #     misc_df = misc_df.drop(['xGF', 'xGA', 'E+/-'], axis=1)
        # except:
        #     pass
        # misc_df.columns  =  [
        #                         'Player', 'Tm', 'GC', 'G_PG', 'A_PG', 'PTS_PG', 'GC_PG',
        #                         'PIM_PG', 'S_PG', 'G_ADJ', 'A_ADJ', 'PTS_ADJ', 'GC_ADJ', 'TGF', 'PGF', 
        #                         'TGA', 'PGA', 'OPS', 'DPS', 'Att.'
        #                     ]
        # raw_tables['misc'] = misc_df

        # Full Dataframe
        full_data = reduce(
            lambda  left,right: pd.merge(
                left,right,on=['Player', 'Tm'],
                how='outer'
            ), [ raw_tables[x] for x in raw_tables.keys() ]
        ).drop_duplicates(subset='Player', keep='first').dropna(subset=['Pos'])
        
        full_data = full_data.replace({
            'C': 'F', 
            'LW': 'F', 
            'RW': 'F', 
            'W': 'F', 
            'NAN': np.nan, 
            '': np.nan
        }).dropna()
        
        # --- Calculate Fantasy Points ---

        g_pts = 3
        a_pts = 2
        pp_pts = 1
        sh_pts = 2
        hat_pts = 1
        sog_pts = 0.1
        hts_pts = 0.1
        blk = 0.2
        
        full_data['FTSY_PTS'] = round(sum([
            (full_data['G'].astype(int)*g_pts), 
            (full_data['A'].astype(int)*a_pts), 
            (full_data['PP_G'].astype(int)*pp_pts), 
            (full_data['SH_G'].astype(int)*sh_pts), 
            # (full_data['HAT']*hat_pts), 
            (full_data['S'].astype(int)*sog_pts), 
            (full_data['HIT'].astype(int)*hts_pts), 
            (full_data['BLK'].astype(int)*blk)
        ]),1)

        return full_data
    
    def get_sec(self, time_str):
        """Get seconds from minutes and seconds."""
        m, s = time_str.split(':')
        return float(int(m) * 60 + int(s))
    
    def get_complete(self):
        complete_df = pd.concat(self.season_datasets.values()).drop(['Rk'], axis=1)
        
        for col in complete_df:
            first = complete_df.loc[0, col].tolist()[0]
            try:
                float(first)
                complete_df[col] = complete_df[col].astype(float)
            except:
                complete_df[col] = complete_df[col].astype(str)
        
        time_cols = ['ATOI', 'TOI/60', 'TOI(EV)', 'Shift', 'TOI_EVEN', 'TOI_PP', 'TOI_SH']
        for col in time_cols:
            complete_df[col] = complete_df[col].apply(lambda time_str: self.get_sec(time_str))
        
        return complete_df.reset_index(drop=True)
    
    def __repr__(self):
        return repr(self.complete_dataset)

## Initialize SeasonData Object

Getting player data from 2008 to the present day would be the ideal dataset, but I keep getting an error:
``` Python
HTTPError: HTTP Error 429: Too Many Requests
```
To get arround this, I will be using a smaller dataset.

In [ ]:
years = list(range(2014, datetime.date.today().year))
data = SeasonsData(years)

Grabbing 2014 season data
1 fails
2 fails
3 fails


## Test Attributes and Methods

In [ ]:
data.shape

59 columns

In [ ]:
data.complete_dataset.query('Player == "CONNOR MCDAVID"')

# Preprocess Data

In [ ]:
dataset = data.complete_dataset

# Inputs
X = dataset.drop(['Player', 'Tm'], axis=1)

# Output
y = dataset['FTSY_PTS']

## Preprocess X

### Drop Variables associated with FTSY_PTS

In [ ]:
to_drop = [
    'G', 'A', 'PP_G','SH_G', 'GW_G', 'EV_G', 'PS',
    'PP_A', 'SH_A', 'BLK','HIT', 'EV_A', 'PTS'
]
X = X.drop(to_drop, axis=1)

### Get Dummies For Pos Column

In [ ]:
posd = pd.get_dummies(X.Pos)
X = pd.concat([X, posd],axis=1).drop(['Pos'], axis=1)

### Add Polynomial Features

In [ ]:
degree = 2
for col in X:
    X[col+f'_{degree}'] = X[col] ** degree

### Drop Highly Correlated Columns

In [ ]:
sns.heatmap(X.corr().abs())

In [ ]:
# Drop highly correlated values
cor_matrix = X.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] >= 0.8)]
X = X.drop(to_drop, axis=1)

print('Dropped: \n', to_drop)

### Normalize Data

In [ ]:
X_norm = preprocessing.normalize(X)
X = pd.DataFrame(X_norm, columns =list(X.columns))

## Preprocess Y

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.hist(y)
ax1.set_title('Not Log Treated')
ax2.hist(np.log(y))
ax2.set_title('Log Treated')

In [ ]:
y = np.log(y)

# OLS Model

## Split Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Fit Model

In [ ]:
model_ols = sm.OLS(y_train, sm.add_constant(X_train)).fit(cov_type='HC3')
model_ols.summary()

In [ ]:
y_pred = model_ols.predict(sm.add_constant(X_test))

y_test_real = np.exp(y_test)
y_pred_real = np.exp(y_pred)

mae = mean_absolute_error(y_test_real, y_pred_real)
mse = mean_squared_error(y_test_real, y_pred_real)
rmse = mean_squared_error(y_test_real, y_pred_real, squared=False)

print(f"Mean Absolute Error: {round(mae, 2)}")
print(f"Mean Squared Error: {round(mse, 2)}")
print(f"Root Mean Squared Error: {round(rmse, 2)}")

# WLS Model

## Calculate Weights

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(10,10))

cols = X.columns
col_idx = 0
for i1 in range(5):
    for i2 in range(5):
        ax[i1,i2].scatter(X[cols[col_idx]], y)
        ax[i1,i2].set_title(cols[col_idx])
        col_idx+=1
        
fig.tight_layout()

In [ ]:
weights = 1 / smf.ols('model_ols.resid.abs() ~ model_ols.fittedvalues', data=X_train).fit().fittedvalues**3

## Fit Model

In [ ]:
model_wls = sm.WLS(y_train, sm.add_constant(X_train), weights=weights).fit()
model_wls.summary()

In [ ]:
y_pred = model_wls.predict(sm.add_constant(X_test))

y_test_real = np.exp(y_test)
y_pred_real = np.exp(y_pred)

mae = mean_absolute_error(y_test_real, y_pred_real)
mse = mean_squared_error(y_test_real, y_pred_real)
rmse = mean_squared_error(y_test_real, y_pred_real, squared=False)

print(f"Mean Absolute Error: {round(mae, 2)}")
print(f"Mean Squared Error: {round(mse, 2)}")
print(f"Root Mean Squared Error: {round(rmse, 2)}")

# Optimized WLS *(Insignificant Values Removed)*

## Remove Insignificant Variables

In [ ]:
to_drop = (
    list(model_wls.pvalues[model_wls.pvalues > 0.05].index)
                           +
    ['Shift', 'ATOI', 'TOI_SH', 'GF/60_PP_2', 'GA/60_PP']
)
X_opt = X.drop(to_drop, axis=1)


print('Dropped: \n', to_drop)

## Split New Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_opt, y, test_size=0.2, random_state=42)

## Recalculate Weights

In [ ]:
model_ols_opt = sm.OLS(y_train, sm.add_constant(X_train)).fit(cov_type='HC3')
weights = 1 / smf.ols('model_ols_opt.resid.abs() ~ model_ols_opt.fittedvalues', data=X_train).fit().fittedvalues**4

In [ ]:
model_opt = sm.WLS(y_train, sm.add_constant(X_train), weights=weights).fit()
model_opt.summary()

In [ ]:
y_pred = model_opt.predict(sm.add_constant(X_test))

y_test_real = np.exp(y_test)
y_pred_real = np.exp(y_pred)

mae = mean_absolute_error(y_test_real, y_pred_real)
mse = mean_squared_error(y_test_real, y_pred_real)
rmse = mean_squared_error(y_test_real, y_pred_real, squared=False)

print(f"Mean Absolute Error: {round(mae, 2)}")
print(f"Mean Squared Error: {round(mse, 2)}")
print(f"Root Mean Squared Error: {round(rmse, 2)}")

# Model Choice

In [ ]:
p_wls = model_wls.pvalues.to_frame().reset_index().rename({'index': 'Models', 0:'WLS Model'}, axis=1).round(2)
p_opt = model_opt.pvalues.to_frame().reset_index().rename({'index': 'Models', 0:'OPT Model'}, axis=1).round(2)

p_table = p_wls.merge(p_opt, on='Models', how='left').sort_values(by='OPT Model').set_index('Models').T.reset_index()

fig, ax = plt.subplots(figsize=(30,0.3))
ax.table(cellText=p_table.values, colLabels=p_table.columns)
ax.axis('off')
ax.set_title('P-Values')

| Metrics  | WLS Model  | Optimized WLS Model  |
|---|---|---|
| MSE  | <span style="color: green;">11.43</span>  | <span style="color: red;">14.17</span>  | 
| RMSE  | <span style="color: green;">16.11</span> | <span style="color: red;">19.88</span>  | 
| R^2  | <span style="color: green;">0.924</span>  | <span style="color: red;">0.885</span>  | 
----------------------------
*Note: The WLS Model has a many values that are statistically insignificant

# Model Summary